In [ ]:
import os
import pandas as pd
import numpy as np
from fancyimpute import KNN
import matplotlib.pyplot as plt
from scipy.stats import chi2_contingency
import seaborn as sns
from ggplot import *
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
import scipy.stats

In [ ]:
os.chdir("D:\DS_New\Project 2")

In [ ]:
df = pd.read_csv("Train_data.csv")
test = pd.read_csv("Test_data.csv")

In [ ]:
#Univariate Analysis
df = df.drop('phone number', axis = 1)
df['area code'] = df['area code'].astype('object')

In [ ]:
#Assigning levels to categorical variables
for i in range(0, df.shape[1]):
    if(df.iloc[:,i].dtypes == 'object'):
        df.iloc[:,i] = pd.Categorical(df.iloc[:,i])
        df.iloc[:,i] = df.iloc[:,i].cat.codes 
        df.iloc[:,i] = df.iloc[:,i].astype('object')

In [ ]:
#Missing Value Analysis
miss_val = pd.DataFrame(df.isnull().sum())
miss_val = miss_val.reset_index()
miss_val = miss_val.rename(columns = {'index': 'Predictors', 0: 'Missing_Percentage'})
miss_val['Missing_Percentage'] = (miss_val['Missing_Percentage']/len(df))*100
miss_val = miss_val.sort_values('Missing_Percentage', ascending = False).reset_index(drop = True)
print(miss_val)

In [ ]:
num_nm = []
cat_nm = []
for i in range(0, df.shape[1]):
        if(df.iloc[:,i].dtypes == 'object'):
            cat_nm.append(df.columns[i])
        else:
            num_nm.append(df.columns[i])

In [ ]:
#Boxplot
ggplot(df, aes(x = 'Churn', y = 'account length', fill = 'Churn')) +\
  geom_boxplot(fill = 'Churn') +\
  ylab("Account Length") + xlab("Churn") + ggtitle("Outlier Analysis")

In [ ]:
#Histogram
ggplot(df, aes(x = 'account length')) + geom_histogram(fill="DarkSlateBlue", colour = "black") +\
    geom_density() +\
    theme_bw() + xlab("Account Length") + ylab("Frequency")  +\
    theme(text=element_text(size=20))

In [ ]:
#Replacing Outliers with NA
for i in num_nm:
     print(i)
     q75, q25 = np.percentile(df.loc[:,i], [75 ,25])
     iqr = q75 - q25
     min = q25 - (iqr*1.5)
     max = q75 + (iqr*1.5)
     print(min)
     print(max)
     df.loc[df.loc[:,i] < min,i] = np.nan
     df.loc[df.loc[:,i] > max,i] = np.nan

In [ ]:
df_test = df

In [ ]:
#Testing Best Method
#Please uncomment and comment as necessary to test
df_test['total day minutes'].loc[70] 
#Original = 241.8
#With Mean = 179.92
#With Median = 179.4
#With KNN = 207.02
df_test['total day minutes'].loc[70] = np.nan
df_test['total day minutes'] = df_test['total day minutes'].fillna(df_test['total day minutes'].mean())
df_test['total day minutes'] = df_test['total day minutes'].fillna(df_test['total day minutes'].median())
df_test = pd.DataFrame(KNN(k = 3).complete(df_test), columns = df_test.columns)

In [ ]:
#Imputing the NaN values with KNN -- Chosen
df = pd.DataFrame(KNN(k = 3).complete(df), columns = df.columns)
#Median
#for i in num_nm:
#    df.loc[:,i]= df.loc[:,i].fillna(df.loc[:,i].median())

In [ ]:
#Correlation Heatmap
num_dt = df.loc[:, num_nm]
f, ax = plt.subplots(figsize=(7,5))
corr_matrix = num_dt.corr()
sns.heatmap(corr_matrix, mask=np.zeros_like(corr_matrix, dtype=np.bool), cmap=sns.diverging_palette(220, 10, as_cmap=True),
            square=True, ax=ax)

In [ ]:
df['Churn'] = df['Churn'].replace(0, 'No')
df['Churn'] = df['Churn'].replace(1, 'Yes')

In [ ]:
#Assessing Variable Importance using Random Forest
X_data = df.iloc[:,0:19]
Y_data = df.iloc[:,19]
rf = RandomForestClassifier(n_estimators = 1000).fit(X_data , Y_data) 
feature_importances = pd.DataFrame(rf.feature_importances_, index = X_data.columns,
                                    columns=['importance']).sort_values('importance',ascending=False)
print(feature_importances)

In [ ]:
#Chi-Square Test of Independence
for i in cat_nm:
    print(i)
    chi2, p, dof, ex = chi2_contingency(pd.crosstab(df['Churn'], df[i]))
    print(p)

In [ ]:
df = df.drop(['area code','account length',
              'total night charge', 'total day charge','total eve charge','total intl charge',
             'total day calls','total eve calls','total night calls'],axis= 1)

In [ ]:
num_nm = []
cat_nm = []
for i in range(0, df.shape[1]):
        if(df.iloc[:,i].dtypes == 'object'):
            cat_nm.append(df.columns[i])
        else:
            num_nm.append(df.columns[i])

In [ ]:
#Normalization
for i in num_nm:
    df.loc[:,i] = (df.loc[:,i] - np.min(df.loc[:,i]))/(np.max(df.loc[:,i]) - np.min(df.loc[:,i]))

In [ ]:
#Cleaning Test Data
test = test.drop(['phone number','area code','account length',
              'total night charge', 'total day charge','total eve charge','total intl charge',
             'total day calls','total eve calls','total night calls'],axis= 1)
for i in range(0, test.shape[1]):
    if(test.iloc[:,i].dtypes == 'object'):
        test.iloc[:,i] = pd.Categorical(test.iloc[:,i])
        test.iloc[:,i] = test.iloc[:,i].cat.codes 
        test.iloc[:,i] = test.iloc[:,i].astype('object')

num_nm = []
cat_nm = []
for i in range(0, test.shape[1]):
        if(test.iloc[:,i].dtypes == 'object'):
            cat_nm.append(test.columns[i])
        else:
            num_nm.append(test.columns[i])

for i in num_nm:
     print(i)
     q75, q25 = np.percentile(test.loc[:,i], [75 ,25])
     iqr = q75 - q25
     min = q25 - (iqr*1.5)
     max = q75 + (iqr*1.5)
     print(min)
     print(max)
     test.loc[test.loc[:,i] < min,i] = np.nan
     test.loc[test.loc[:,i] > max,i] = np.nan

#Imputing with Median
#for i in num_nm:
#    test.loc[:,i]= test.loc[:,i].fillna(test.loc[:,i].median())

#Imputing with KNN
test = pd.DataFrame(KNN(k = 3).complete(test), columns = test.columns)

for i in num_nm:
    if(i != 'number vmail messages'):
        test.loc[:,i] = (test.loc[:,i] - np.min(test.loc[:,i]))/(np.max(test.loc[:,i]) - np.min(test.loc[:,i]))
    

In [ ]:
test['Churn'] = test['Churn'].replace(0, 'No')
test['Churn'] = test['Churn'].replace(1, 'Yes')
df['Churn'] = df['Churn'].replace(0, 'No')
df['Churn'] = df['Churn'].replace(1, 'Yes')

In [ ]:
X_train = df.iloc[:,0:10]
Y_train = df.iloc[:,10]
X_test = test.iloc[:,0:10]
Y_test = test.iloc[:,10]

In [ ]:
#Decision Tree
DT_model = tree.DecisionTreeClassifier(criterion = "entropy").fit(X_train, Y_train)
DT_pred = DT_model.predict(X_test)


In [ ]:
#Evaluating our Decision Tree Model
CM = pd.crosstab(Y_test,DT_pred)
AS = accuracy_score(Y_test, DT_pred)*100
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FNR = FN/(FN+TP)
print(CM)
print(FNR)
print(AS)


In [ ]:
#Random Forest
RF_model = RandomForestClassifier(n_estimators = 1000).fit(X_train, Y_train)
RF_pred = RF_model.predict(X_test)


In [ ]:
#Evaluating Random Forest
CM = pd.crosstab(Y_test,RF_pred)
AS = accuracy_score(Y_test, RF_pred)*100
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FNR = FN/(FN+TP)
print(CM)
print(FNR)
print(AS)

In [ ]:
#Logistic Regression
#Preparing the data
test['Churn'] = test['Churn'].replace('No', 0)
test['Churn'] = test['Churn'].replace('Yes', 1)
df['Churn'] = df['Churn'].replace('No', 0)
df['Churn'] = df['Churn'].replace('Yes', 1)

df_logit = pd.DataFrame(df['Churn'])
df_logit = df_logit.join(df[num_nm])

for i in cat_nm:
    if(i != 'Churn'):
        temp = pd.get_dummies(df[i], prefix = i)
        df_logit = df_logit.join(temp)
        
test_logit = pd.DataFrame(test['Churn'])
test_logit = test_logit.join(test[num_nm])

for i in cat_nm:
    if(i != 'Churn'):
        temp = pd.get_dummies(test[i], prefix = i)
        test_logit = test_logit.join(temp)


In [ ]:
#Creating a Logistic Regession model
LogR_model = sm.Logit(df_logit['Churn'], df_logit.iloc[:,1:63]).fit()
test_logit['Actual Probability'] = LogR_model.predict(test_logit.iloc[:,1:63])
test_logit['Actual Value'] = 1
test_logit.loc[test_logit.loc[:,'Actual Probability'] < 0.5, 'Actual Value'] = 0

In [ ]:
#Evaluating the Logistic Regression Model
CM = pd.crosstab(test_logit['Churn'],test_logit['Actual Value'])
AS = accuracy_score(test_logit['Churn'], test_logit['Actual Value'])*100
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FNR = FN/(FN+TP)
print(CM)
print(FNR)
print(AS)


In [ ]:
#KNN
#Preparing the data
test['Churn'] = test['Churn'].replace(0, 'No')
test['Churn'] = test['Churn'].replace(1, 'Yes')
df['Churn'] = df['Churn'].replace(0, 'No')
df['Churn'] = df['Churn'].replace(1, 'Yes')

X_train = df.iloc[:,0:10]
Y_train = df.iloc[:,10]
X_test = test.iloc[:,0:10]
Y_test = test.iloc[:,10]

KNN_model = KNeighborsClassifier(n_neighbors = 3).fit(X_train, Y_train)
KNN_Pred = KNN_model.predict(X_test)

In [ ]:
#Evaluating KNN model
CM = pd.crosstab(Y_test,KNN_Pred)
AS = accuracy_score(Y_test, KNN_Pred)*100
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FNR = FN/(FN+TP)
print(CM)
print(FNR)
print(AS)

In [ ]:
#Naive Bayes
NB_model = GaussianNB().fit(X_train, Y_train)
NB_pred = NB_model.predict(X_test)

In [ ]:
#Evaluating the Naive Bayes model
CM = pd.crosstab(Y_test,NB_pred)
AS = accuracy_score(Y_test, NB_pred)*100
FN = CM.iloc[1,0]
TP = CM.iloc[1,1]
FNR = FN/(FN+TP)
print(CM)
print(FNR)
print(AS)